In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# usar Jena para transformar o CSV da ontologia do banco de dados em um .owl. 
# Importar esse .owl na lib Owlready2 https://pypi.org/project/Owlready2/
# https://stackoverflow.com/questions/13147864/how-to-create-an-ontology-from-raw-data-csv
# https://www.google.com/search?q=how+to+transform+csv+ontologies+in+.owl+with+python&oq=how+to+transform+csv+ontologies+in+.owl+with+python&aqs=chrome..69i57.19783j1j7&gs_lcrp=EgZjaHJvbWUyBggAEEUYOdIBCTE5NzgzajFqN6gCALACAA&sourceid=chrome&ie=UTF-8
# https://www.w3.org/wiki/ConverterToRdf
# http://tarql.github.io/
# Identificar os "identifiers"dentro da anotação livre e colocar eles em uma coluna do dataframe, depois pegar as ontologias associadas a eles e colocar em outra coluna, vetorizar tudo no final

# https://github.com/neuralmind-ai/portuguese-bert
# https://huggingface.co/neuralmind
# https://huggingface.co/neuralmind/bert-base-portuguese-cased


# Compara o NER com as ontologias e ver de da um match. Tomar cuidado para que não insira bias no modelo, pois talvez os textos que contenham um match não necessariamente são os com as maiores notas

In [2]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
from sklearn import preprocessing
# from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR, SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer, r2_score
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import spacy 
%matplotlib inline

In [ ]:
!python -m spacy download pt_core_news_sm 

2022-11-15 21:40:34.867849: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 4.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch

In [ ]:
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads

# model = AutoModelForPreTraining.from_pretrained('neuralmind/bert-base-portuguese-cased')
# tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

In [ ]:
# model2 = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
# input_ids = tokenizer.encode(free_annotations['clean_content'][0], return_tensors='pt')

# with torch.no_grad():
#     outs = model(input_ids)
#     encoded = outs[0][0, 1:-1]  # Ignore [CLS] and [SEP] special tokens
# encoded.numpy()

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


array([[  0.41994292,   0.48326707,  -0.9043782 , ...,  -5.622022  ,
          0.13079238,  -3.2076883 ],
       [ -6.8490324 ,  -5.9790974 ,  -8.455076  , ...,  -9.359343  ,
         -6.419617  , -10.077203  ],
       [ -8.257705  , -10.489056  , -11.35701   , ...,  -9.293404  ,
         -7.558038  , -10.717872  ],
       ...,
       [ -7.9566813 ,  -8.799402  , -10.748509  , ...,  -8.614093  ,
         -6.5400305 ,  -8.779255  ],
       [ -3.4925451 ,  -4.678734  ,  -5.0204206 , ...,  -6.609981  ,
         -4.435259  ,  -7.39447   ],
       [ -9.015351  , -11.736718  , -11.692457  , ..., -12.515277  ,
         -7.6735983 , -14.410841  ]], dtype=float32)

In [ ]:
# nlp = spacy.blank('pt')
# doc = nlp(annotations['content'][0])
# token0 = doc[0]
# print(token0.like_url)

In [ ]:
# nlp2 = spacy.load('pt_core_news_sm')
# doc2 = nlp2(annotations['content'][0])
# for ent in doc2.ents:
#     print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
# annotations['content'][0]

In [ ]:
# !pip install spacy

In [ ]:
# !pip install quilt
# !quilt install ResidentMario/missingno_data
# !pip install lxml
# !pip install beautifulsoup4
# !pip install chardet

vectorized_with_ner = pd.read_csv('/content/drive/My Drive/TCC/csv/completo/vectorized_with_ner.csv')

In [ ]:
# annotations = pd.read_csv('/content/drive/My Drive/TCC/csv/completo/annotation_free_annotation.csv', encoding="utf-8")
# tudo = pd.read_csv('/content/drive/My Drive/TCC/csv/completo/todos.csv', encoding="utf-8")

In [3]:
free_annotations = pd.read_csv('/content/drive/My Drive/TCC/csv/completo/free_annotations.csv')
semantic_annotations = pd.read_csv('/content/drive/My Drive/TCC/csv/completo/semantic_annotation_ontology.csv')
ontology_full = pd.read_csv('/content/drive/My Drive/TCC/csv/completo/ontology_full.csv')
notas = pd.read_csv('/content/drive/My Drive/TCC/csv/completo/notas.csv', usecols=[0,2,4,6,8,10,12])

In [4]:
ontology_full.head()

,identifier,label
0,1 Prof. Domingos Antena,Preenchido
1,1 Prof. Domingos Antena,Nulo
2,1 Prof. Domingos Antena,Tipo de Sujeito
3,1 Prof. Domingos Antena,subjuntivo
4,1 Prof. Domingos Antena,perfeito do indicativo


In [ ]:
notas.columns

Index(['Nome / Sobrenome', 'Endereço de email',
       'Resenha de "Senhora", de José de Alencar',
       'Resenha de "Iaiá Garcia", de Machado de Assis',
       'Resenha de "Memórias póstumas de Brás Cubas", de Machado de Assis',
       'Resenha de "Quincas Borba", de Machado de Assis',
       'Resenha de "Dom Casmurro", de Machado de Assis'],
      dtype='object')

In [ ]:
notas.rename(columns = {'Endereço de email': 'email'}, inplace = True)
# notas.head(1)
print('')
# notas.rename(columns = {'Resenha de "Iaiá Garcia", de Machado de Assis': 'Iaiá Garcia'})
# notas.rename(columns = {'Resenha de "Memórias póstumas de Brás Cubas", de Machado de Assis': 'Memórias póstumas de Brás Cubas'})
# notas.rename(columns = {'Resenha de "Quincas Borba", de Machado de Assis': 'Quincas Borba'})
# notas.rename(columns = {'Resenha de "Senhora", de José de Alencar': 'Senhora'})
# notas.rename(columns = {'Resenha de "Dom Casmurro", de Machado de Assis': 'Dom Casmurro'})



In [ ]:
notas.rename(columns = {'Resenha de "Senhora", de José de Alencar': 'José Martiniano de Alencar – Senhora'}, inplace = True)
print('')

In [ ]:
notas.rename(columns = {'Resenha de "Iaiá Garcia", de Machado de Assis': 'Joaquim Maria Machado de Assis – Iaiá Garcia:'}, inplace = True)
# notas.head(1)
print('')

In [ ]:
notas.rename(columns = {'Resenha de "Memórias póstumas de Brás Cubas", de Machado de Assis': 'Joaquim Maria Machado de Assis – Memórias Póstumas de Brás Cubas'}, inplace = True)
# notas.head(1)
print('')

In [ ]:
notas.rename(columns = {'Resenha de "Quincas Borba", de Machado de Assis': 'Quincas Borba, de Machado de Assis'}, inplace=True)
# notas.head(1)
print('')

In [ ]:
notas.rename(columns = {'Resenha de "Dom Casmurro", de Machado de Assis': 'Dom Casmurro – Joaquim Maria Machado de Assis'}, inplace=True)
# notas.head(1)
print('')

In [ ]:
notas.replace('-',0, inplace=True)

In [ ]:
# notas.drop(labels='Unnamed: 1', axis=1)
notas.head()

,Nome / Sobrenome,email,José Martiniano de Alencar – Senhora,Joaquim Maria Machado de Assis – Iaiá Garcia:,Joaquim Maria Machado de Assis – Memórias Póstumas de Brás Cubas,"Quincas Borba, de Machado de Assis",Dom Casmurro – Joaquim Maria Machado de Assis
0,Allan Dayeh Serrão (13102434),allandayehs@yahoo.com.br,10,10,0,0,0
1,Anna Viana Salviato (13102436),annasalviato@hotmail.com,10,10,10,10,9
2,Caroline Batista de Britto Gallo (10106304),carolinegallo.marykay@bol.com.br,0,0,0,0,0
3,Caroline Dyana de Sousa Maes (13102337),carolinesmaes@hotmail.com,"8,5",0,0,"7,5",0
4,Debora Machado Gonçalves (13102440),goncalvesmdebora@gmail.com,9,0,9,9,0


In [ ]:
# notas['José Martiniano de Alencar – Senhora']

In [ ]:
free_annotations.head(1)
# options = ['Science', 'Commerce'] 
    
# # selecting rows based on condition 
# rslt_df = dataframe[dataframe['Stream'].isin(options)] 

,excerpt,content,title,email
0,pertendo,"<p>\n\tpertendo: pertender, pertendente, prete...",Poemas de Gregório de Matos,veronicaamato@gmail.com


In [ ]:
free_annotations['title'].unique()

array(['Poemas de Gregório de Matos',
       'Obras poéticas de Inácio José de Alvarenga Peixoto',
       'José de Anchieta – Auto representado na Festa de São Lourenço',
       'Auto representado na festa de São Lourenço, de José de Anchieta',
       'Sermão da Sexagésima, de Padre António Vieira',
       'Statira, e Zoroastes, de Lucas José de Alvarenga',
       'Joaquim Maria Machado de Assis – Notícia da atual literatura brasileira',
       'A Família Agulha, de Luís Guimarães Jr.',
       'José Martiniano de Alencar – Senhora',
       'Joaquim Maria Machado de Assis – Iaiá Garcia:',
       'Joaquim Maria Machado de Assis – Memórias Póstumas de Brás Cubas',
       'Quincas Borba, de Machado de Assis',
       'Dom Casmurro – Joaquim Maria Machado de Assis',
       'Augusto de Carvalho Rodrigues dos Anjos – Obra Completa',
       'A Carteira, de Machado de Assis', 'A carteira – ',
       'A carteira – Joaquim Maria Machado de Assis',
       'Henrique Maximiano Coelho Neto – Edição di

In [ ]:
titles = ['José Martiniano de Alencar – Senhora','Joaquim Maria Machado de Assis – Iaiá Garcia:','Joaquim Maria Machado de Assis – Memórias Póstumas de Brás Cubas','Quincas Borba, de Machado de Assis','Dom Casmurro – Joaquim Maria Machado de Assis']
# dataframe[dataframe['Stream'].isin(options)] 
free_annotations = free_annotations.loc[free_annotations['title'].isin(titles)]
free_annotations

,excerpt,content,title,email
235,Lembrou-se do seu Byron,<p>\n\tA poesia de Lord Byron inspira a segund...,José Martiniano de Alencar – Senhora,annasalviato@hotmail.com
236,"Aurélia Camargo,","<p>\n\tA personagem, em v&aacute;rios momentos...",José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com
237,"Suspeito eu, porém, que a explicação dessa sin...",<p>\n\tEssa passagem me fez lembrar da obra &q...,José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com
238,Ao Leitor,<p>\n\tAchei curioso o fato de que o autor se ...,José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com
239,"Era rica e formosa. // Duas opulências, que s...",<p>\n\tAur&eacute;lia com as duas caracter&iac...,José Martiniano de Alencar – Senhora,unga_@hotmail.com
...,...,...,...,...
700,"Não importa, continuou Capitu;",<p>\n\tInserir aqui o que se quer comentar.</p>\n,Dom Casmurro – Joaquim Maria Machado de Assis,kleber_lz@hotmail.com
701,"Ia a entrar na sala de visitas,",<p>\n\tCorpo da anota&ccedil;&atilde;o que est...,Dom Casmurro – Joaquim Maria Machado de Assis,kleber_lz@hotmail.com
702,"Uma noite destas, vindo da cidade para o Engen...",<p>O curso de Letras &eacute; o melhor da UEMA...,Dom Casmurro – Joaquim Maria Machado de Assis,lordemanoel@hotmail.com
703,"Continue, disse eu acordando",<p>Qual o tipo de discurso aqui?</p>\n,Dom Casmurro – Joaquim Maria Machado de Assis,lordemanoel@hotmail.com


In [ ]:
# notas['José Martiniano de Alencar – Senhora']

In [ ]:
emails = ['allandayehs@yahoo.com.br',
				'annasalviato@hotmail.com',

				'carolinegallo.marykay@bol.com.br',
				'carolinesmaes@hotmail.com',
				'dalvalobo@terra.com.br',
				'goncalvesmdebora@gmail.com'
				'dedetco@icloud.com',
				'denise_study@hotmail.com',
				'hauchduda@hotmail.com',
				'smonteiro.nani@gmail.com',
				'profelianapaz@gmail.com',
				'lordemanoel@hotmail.com',
				'ge.woolf@gmail.com',
				'gabrielesteues@gmail.com',
				'guto7wolf@gmail.com',
				'helenadiasgrad@gmail.com',
				'iriscostadeandrade@hotmail.com',
				'isamelim74@gmail.com',
				'kleber_lz@hotmail.com',
				'larissamalu@hotmail.com',
				'unga_@hotmail.com',
				'bmagridt@gmail.com',
				'manumariani@gmail.com',
				'marcias@fatma.sc.gov.br',
				'mariungaretti@gmail.com',
				'nathanschmitt@bol.com.br',
				'pedrobouzan@gmail.com',
				'rafaelmsens@gmail.com',
				'samantta.portela@gmail.com',
				'veronicaamato@gmail.com']
# dataframe[dataframe['Stream'].isin(options)] 
free_annotations = free_annotations.loc[free_annotations['email'].isin(emails)]
free_annotations

,excerpt,content,title,email
235,Lembrou-se do seu Byron,<p>\n\tA poesia de Lord Byron inspira a segund...,José Martiniano de Alencar – Senhora,annasalviato@hotmail.com
236,"Aurélia Camargo,","<p>\n\tA personagem, em v&aacute;rios momentos...",José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com
237,"Suspeito eu, porém, que a explicação dessa sin...",<p>\n\tEssa passagem me fez lembrar da obra &q...,José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com
238,Ao Leitor,<p>\n\tAchei curioso o fato de que o autor se ...,José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com
239,"Era rica e formosa. // Duas opulências, que s...",<p>\n\tAur&eacute;lia com as duas caracter&iac...,José Martiniano de Alencar – Senhora,unga_@hotmail.com
...,...,...,...,...
700,"Não importa, continuou Capitu;",<p>\n\tInserir aqui o que se quer comentar.</p>\n,Dom Casmurro – Joaquim Maria Machado de Assis,kleber_lz@hotmail.com
701,"Ia a entrar na sala de visitas,",<p>\n\tCorpo da anota&ccedil;&atilde;o que est...,Dom Casmurro – Joaquim Maria Machado de Assis,kleber_lz@hotmail.com
702,"Uma noite destas, vindo da cidade para o Engen...",<p>O curso de Letras &eacute; o melhor da UEMA...,Dom Casmurro – Joaquim Maria Machado de Assis,lordemanoel@hotmail.com
703,"Continue, disse eu acordando",<p>Qual o tipo de discurso aqui?</p>\n,Dom Casmurro – Joaquim Maria Machado de Assis,lordemanoel@hotmail.com


In [ ]:
len(free_annotations)

385

In [ ]:
# student = notas.where(notas['email']=='manumariani@gmail.com').dropna()
# student

,Nome / Sobrenome,email,"Resenha de ""Senhora"", de José de Alencar",Joaquim Maria Machado de Assis – Iaiá Garcia:,Joaquim Maria Machado de Assis – Memórias Póstumas de Brás Cubas,"Quincas Borba, de Machado de Assis",Dom Casmurro – Joaquim Maria Machado de Assis
18,Manuella Mariani (13103754),manumariani@gmail.com,0,0.0,0.0,0,0.0


In [ ]:
# x = pd.merge(free_annotations, notas,on=["email"])
# x.head()





In [ ]:
# def merge_results(title, email):
#   print(email.values)
#   # print(title,email)
#   # return content
  
#   # student = notas.where(notas['email']== email).dropna()
#   # print(student)


In [ ]:
# free_annotations['clean_content'] = free_annotations['content'].apply(lambda x: clean_content(str(x)))
# x = free_annotations.apply(lambda x: merge_results(free_annotations['title'], free_annotations['email']))

In [ ]:
free_annotations['notas'] = None
for row in free_annotations.itertuples():
  try:  
    student = notas.where(notas['email']== row.email).dropna()
    e = student[row.title] 
    free_annotations['notas'][row.Index] = float(e.values.item())
  except:
      pass


In [ ]:
free_annotations.head()

,excerpt,content,title,email,notas
235,Lembrou-se do seu Byron,<p>\n\tA poesia de Lord Byron inspira a segund...,José Martiniano de Alencar – Senhora,annasalviato@hotmail.com,10.0
236,"Aurélia Camargo,","<p>\n\tA personagem, em v&aacute;rios momentos...",José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com,10.0
237,"Suspeito eu, porém, que a explicação dessa sin...",<p>\n\tEssa passagem me fez lembrar da obra &q...,José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com,10.0
238,Ao Leitor,<p>\n\tAchei curioso o fato de que o autor se ...,José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com,10.0
239,"Era rica e formosa. // Duas opulências, que s...",<p>\n\tAur&eacute;lia com as duas caracter&iac...,José Martiniano de Alencar – Senhora,unga_@hotmail.com,10.0


In [ ]:
# free_annotations['notas'].head()

In [ ]:
# results

In [ ]:
# def merge_results(email, title):
#   email = str(email.values)
#   # print(type(email))
#   # student = notas.where(notas['email'] == email).dropna()
#   # print(email)
#   # print(type(str(email.values)))

#   # free_annotations[]
#   # print(student)
#   # print(content)


In [ ]:
# df.apply(lambda x: f(x.col_1, x.col_2), axis=1)
# free_annotations['email'].apply(lambda x: merge_results(free_annotations['email'],free_annotations['title']))

In [ ]:
free_annotations.head()

,excerpt,content,title,email
235,Lembrou-se do seu Byron,<p>\n\tA poesia de Lord Byron inspira a segund...,José Martiniano de Alencar – Senhora,annasalviato@hotmail.com
236,"Aurélia Camargo,","<p>\n\tA personagem, em v&aacute;rios momentos...",José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com
237,"Suspeito eu, porém, que a explicação dessa sin...",<p>\n\tEssa passagem me fez lembrar da obra &q...,José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com
238,Ao Leitor,<p>\n\tAchei curioso o fato de que o autor se ...,José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com
239,"Era rica e formosa. // Duas opulências, que s...",<p>\n\tAur&eacute;lia com as duas caracter&iac...,José Martiniano de Alencar – Senhora,unga_@hotmail.com


In [ ]:
# free_annotations.assign(result=lambda x: join_results(str(x)))

In [ ]:
len(ontology_full)

100000

In [ ]:
# semantic_annotations.head()

In [ ]:
# free_annotations.head(10)

In [ ]:
# free_annotations['title'][9]

In [ ]:
# x = clean_content( free_annotations['content'][9])
# x

In [ ]:
# ontology.head()

,identifier,label
0,Aires,Autor
1,Aires,Pessoa
2,Natividade,Pessoa
3,Sertão,Região
4,Gustavo,Pessoa


In [ ]:
len(ontology_full)

100000

In [ ]:
# ontology_full.drop_duplicates(inplace=True)


In [ ]:
# free_annotations['clean_content'][50]

In [ ]:
# content = free_annotations['clean_content'][12]
# content.lower()
# if 'poeta' in content:
#   print('contem')
# content.find('poeta')

'o poeta faz referencia aos perços exorbitantes que eram pagos por mercadorias no mercado'

In [ ]:
# (ontology_full['identifier'].eq('Coelho Neto')).any()

False

In [ ]:
# free_annotations.dropna(inplace=True)
# semantic_annotations.dropna(inplace=True)

In [ ]:
# len(free_annotations.dropna())
# len(semantic_annotations.dropna())

1466

In [ ]:
# import chardet 

# enco = chardet.detect(annotations['content'][13].encode())
# print(enco)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [ ]:
# annotations.head(25)

In [ ]:
# tudo.head()

,annotation_id,ref_user,excerpt,name,class_resource,label_class,label_domain,propertie_resource,prop_range,max_cardinality
0,65291,5792,"A minha musa, a minha pobre musa, // De riso ...",Principal 2016-06-23,http://literaturabrasileira.ufsc.br/kb#Eu-Poético,Eu Poético,Gênero,http://literaturabrasileira.ufsc.br/kb#Gênero,http://www.w3.org/2000/01/rdf-schema#Literal,1
1,65291,5792,"A minha musa, a minha pobre musa, // De riso ...",Principal 2016-06-23,http://literaturabrasileira.ufsc.br/kb#Eu-Poético,Eu Poético,Gênero,http://literaturabrasileira.ufsc.br/kb#Gênero,http://www.w3.org/2000/01/rdf-schema#Literal,1
2,65394,5792,Sou rapariga da aldeia; // Cercam-me os moços...,Principal 2016-06-23,http://literaturabrasileira.ufsc.br/kb#Eu-Poético,Eu Poético,Gênero,http://literaturabrasileira.ufsc.br/kb#Gênero,http://www.w3.org/2000/01/rdf-schema#Literal,1
3,65394,5792,Sou rapariga da aldeia; // Cercam-me os moços...,Principal 2016-06-23,http://literaturabrasileira.ufsc.br/kb#Eu-Poético,Eu Poético,Gênero,http://literaturabrasileira.ufsc.br/kb#Gênero,http://www.w3.org/2000/01/rdf-schema#Literal,1
4,65470,5792,Caíra o sol no horizonte! // A rapariga trave...,Principal 2016-06-23,http://literaturabrasileira.ufsc.br/kb#Eu-Poético,Eu Poético,Gênero,http://literaturabrasileira.ufsc.br/kb#Gênero,http://www.w3.org/2000/01/rdf-schema#Literal,1


In [ ]:
# annotations['content'].head(25)

In [ ]:
# teste = annotations['content'][11]
# tst1 = teste.encode('ascii').decode('UTF-8')
# tst1

'<p>\n\tProcure por informa&ccedil;&otilde;es sobre a ret&oacute;rica cl&aacute;ssica. Veja, por exemplo: Barthes, Roland. &quot;A Ret&oacute;rica Antiga&quot; in <em>Pesquisas de Ret&oacute;rica</em>, Petr&oacute;polis: Vozes, 1975.</p>\n'

In [ ]:
from bs4 import BeautifulSoup

# Utilizando o beautiful soup para pegar o texto que esta dentro do html da anotacao
# soup = BeautifulSoup(teste, 'lxml')

# tst2 = soup.get_text()
# TODO: usar regex para formatar as quebras de linhas ou pesquisar se o bs4 faz isso
# tst2

In [ ]:
# tst2 = tst2.strip('\n\t')

In [ ]:
# tst2

'Procure por informações sobre a retórica clássica. Veja, por exemplo: Barthes, Roland. "A Retórica Antiga" in Pesquisas de Retórica, Petrópolis: Vozes, 1975.'

In [ ]:
ontology_full['identifier'].drop_duplicates(inplace=True)

In [ ]:
links = count_links(free_annotations['content'])

In [ ]:
links

45

In [ ]:
nlp_blank = spacy.blank('pt')
def remove_links(contents):
  for content in contents:
    doc = nlp_blank(content)
    for token in doc:
      if(token.like_url):
        # print(content)
        return contents
  # return count

In [ ]:
# removes = remove_links(free_annotations['clean_content'])
# len(removes)

In [ ]:
links2 = count_links(removes)
links2

43

In [ ]:
def clean_content(content):
  soup =  BeautifulSoup(content, 'lxml')
  clean_content = soup.get_text().strip('\n\t\r')
  return clean_content

In [ ]:
nlp_blank = spacy.blank('pt')
def count_links(contents):
  count = 0 
  for content in contents:
    doc = nlp_blank(content)
    for token in doc:
      if(token.like_url):
        count+=1
  return count

In [ ]:
# for ent in doc2.ents:
#     print(ent.text, ent.start_char, ent.end_char, ent.label_)
# TODO: colocar uma nova coluna no dataframe que conta a quantidade de NER que aparecem no texto, e fazer a contagem nessa coluna com base na quantidade de matches que dão nas ontologias
nlp_ner = spacy.load('pt_core_news_sm')
def check_ner(content):
  ents = []
  doc = nlp_ner(content)
  for ent in doc.ents:
    ent = ent.vector
    # print(ent)
    ents.append(ent.ravel())
    # print(ent)
    # np.asarray(someListOfLists, dtype=np.float32)
  return float(len(ents))


In [ ]:
# Funcao para contar a quantidade de part of speach tagging que tem contida no texto e criar uma coluna no dataframe com elas
nlp_pos = spacy.load('pt_core_news_sm')
def count_pos(content):
  pos = []
  doc = nlp_pos(content)
  count = doc.count_by(spacy.attrs.POS)
  # print(count)
  for k, v in count.items():
    free_annotations[doc.vocab[k].text] = float(v)
    # print(doc.vocab[k].text, ' | ', v)

In [ ]:
nlp_vec =  spacy.load('pt_core_news_sm')
def vectorize_sentence(content):
  vec = nlp_vec(content)
  # print(vec.vector)
  # print(vec.vector_norm)
  # print(column_name)
  # free_annotations['vectorized_content'] = vec.vector
  return vec.vector_norm
  # print(vec.vector)

In [ ]:
def find_ontology(content):
  ontos = ontology_full['identifier']
  found_ontos = []
  for onto in ontos:
    if onto in content:
      found_ontos.append(onto)
  return found_ontos
  
    # if element.lower() in content.lower():
      # print('contem')

In [ ]:
bertimbau = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
tokenizer.model_max_length=1024
model = AutoModelForPreTraining.from_pretrained('neuralmind/bert-base-portuguese-cased')

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForPreTraining were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newl

In [ ]:
# model2 = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
# input_ids = tokenizer.encode('Tinha uma pedra no meio do caminho.', return_tensors='pt')

# with torch.no_grad():
#     outs = model(input_ids)
#     encoded = outs[0][0, 1:-1]  # Ignore [CLS] and [SEP] special tokens

# encoded
# bertimbau = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
# tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
# tokenizer.model_max_length=1024
# model = AutoModelForPreTraining.from_pretrained('neuralmind/bert-base-portuguese-cased')
def vectorize_sentence_bert(content):
  try:
    vec = tokenizer.encode(content,return_tensors='pt')  
    # print(type(vec))
    with torch.no_grad():
      out = model(vec)
      encoded = out[0][0,1:-1]
      encoded = encoded.numpy()
      # print(type(encoded))
      # print(norm(encoded))
      return norm(encoded)
  except:
    pass
    # print(content)


In [ ]:
free_annotations['clean_content'] = free_annotations['content'].apply(lambda x: clean_content(str(x)))

In [ ]:
# words = set()
# def create_set_of_words(content):
#   splited_content = content.split()
#   for word in splited_content:
#     words.add(word)
#     # print(content)
# free_annotations['clean_content'].apply(lambda x: create_set_of_words(str(x)))


235    None
236    None
237    None
238    None
239    None
       ... 
700    None
701    None
702    None
703    None
704    None
Name: clean_content, Length: 385, dtype: object

In [ ]:
# pd_words = pd.DataFrame(columns = words)
# free_annotations =  pd.concat([pd_words, free_annotations], axis=1)


In [ ]:
# free_annotations.head()
free_annotations['vectorized_content_bert']

235    None
236    None
237    None
238    None
239    None
       ... 
700    None
701    None
702    None
703    None
704    None
Name: vectorized_content_bert, Length: 385, dtype: object

In [ ]:
free_annotations['vectorized_content_bert'] = free_annotations['clean_content'].apply(lambda x: vectorize_sentence_bert(str(x)))

In [ ]:
free_annotations['vectorized_excerpt_bert'] = free_annotations['excerpt'].apply(lambda x: vectorize_sentence_bert(str(x)))

In [ ]:
free_annotations['NER'] = free_annotations['clean_content'].apply(lambda x: check_ner(str(x)))

In [ ]:
free_annotations['NER'].head()

235    [[1.2726889, 2.8953836, -0.7336545, 1.0329155,...
236    [[2.4862916, 2.107809, 1.8457295, -0.040821433...
237    [[1.3252956, 1.9256511, -1.6719532, -2.177884,...
238    [[1.465732, 0.2867495, -3.225106, 1.086696, 2....
239    [[0.77389693, 4.9895525, -2.745476, 6.034116, ...
Name: NER, dtype: object

In [ ]:
free_annotations['clean_content'].apply(lambda x: count_pos(str(x)))

235    None
236    None
237    None
238    None
239    None
       ... 
700    None
701    None
702    None
703    None
704    None
Name: clean_content, Length: 385, dtype: object

In [ ]:
free_annotations['found_ontologies'] = free_annotations['clean_content'].apply(lambda x: find_ontology(str(x)))

In [ ]:
#@title Texto de título padrão
# vector = vectorize_sentence(free_annotations['clean_content'][0],'x')
# vector
free_annotations['vectorized_content'] = free_annotations['clean_content'].apply(lambda x: vectorize_sentence(str(x)))

In [ ]:
free_annotations['vectorized_excerpt'] = free_annotations['excerpt'].apply(lambda x: vectorize_sentence(str(x)))

In [ ]:
free_annotations['NER'].tail()

702    [[1.9339774, -0.37884697, -2.2920675, 0.648855...
703                                                   []
704                                                   []
705    [[1.4808745, -0.6924863, -0.38353747, -2.39398...
706                                                   []
Name: NER, dtype: object

In [ ]:
free_annotations['vectorized_content_bert'].head()

0    None
1    None
2    None
3    None
4    None
Name: vectorized_content_bert, dtype: object

In [ ]:
# free_annotations['vectorized_content'] = free_annotations['clean_content']

In [ ]:
free_annotations['found_ontologies']

235    []
236    []
237    []
238    []
239    []
       ..
700    []
701    []
702    []
703    []
704    []
Name: found_ontologies, Length: 385, dtype: object

In [ ]:
free_annotations.head()

,excerpt,content,title,email,notas,clean_content,DET,NOUN,ADP,PROPN,...,SPACE,ADV,SCONJ,AUX,NUM,SYM,X,INTJ,vectorized_content,vectorized_excerpt
235,Lembrou-se do seu Byron,<p>\n\tA poesia de Lord Byron inspira a segund...,José Martiniano de Alencar – Senhora,annasalviato@hotmail.com,10.0,A poesia de Lord Byron inspira a segunda geraç...,2.0,2.0,1.0,2.0,...,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,10.119796,14.894006
236,"Aurélia Camargo,","<p>\n\tA personagem, em v&aacute;rios momentos...",José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com,10.0,"A personagem, em vários momentos da obra me fe...",2.0,2.0,1.0,2.0,...,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,7.907671,18.088019
237,"Suspeito eu, porém, que a explicação dessa sin...",<p>\n\tEssa passagem me fez lembrar da obra &q...,José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com,10.0,"Essa passagem me fez lembrar da obra ""Romeu e ...",2.0,2.0,1.0,2.0,...,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,8.283214,9.208959
238,Ao Leitor,<p>\n\tAchei curioso o fato de que o autor se ...,José Martiniano de Alencar – Senhora,kleber_lz@hotmail.com,10.0,Achei curioso o fato de que o autor se refere ...,2.0,2.0,1.0,2.0,...,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,9.333219,21.802164
239,"Era rica e formosa. // Duas opulências, que s...",<p>\n\tAur&eacute;lia com as duas caracter&iac...,José Martiniano de Alencar – Senhora,unga_@hotmail.com,10.0,Aurélia com as duas características mais preza...,2.0,2.0,1.0,2.0,...,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,10.603291,8.455762


In [ ]:
# nlp12 = spacy.load('pt_core_news_sm')
# test_doc = nlp12(free_annotations['clean_content'][0])
# pos = test_doc.count_by(spacy.attrs.POS)

In [ ]:
# TODO: Criar uma nova coluna com dados de part-of-speech tagging e criar também uma coluna com as sentenças vetorizadas. Avaliar a possibilidade de usar BERT
# https://spacy.io/usage/linguistic-features

In [ ]:
# Premissa: Nota do professor na atividade seria 10, por ser especialista na área.

In [ ]:
free_annotations['clean_content'].head()

0        pertendo: pertender, pertendente, pretendente
1    Pretende encontrar a salvação nos braços de Je...
2    O poeta assume, neste ponto, que ofende a Deus...
3       Peça do vestuário que se assemelha a um gorro.
4    Descreve, aqui, a indumentária do estudante da...
Name: clean_content, dtype: object

In [ ]:
count =+ count_links(free_annotations['content'])
count
# 1699 links

58

In [ ]:
# annotations['clean_content'].dropna(axis=0,inplace=True)
# free_annotations.head(1)

,excerpt,content,title,email,notas,clean_content,vectorized_content_bert,vectorized_excerpt_bert,NER,DET,...,CCONJ,PRON,SPACE,ADV,SCONJ,AUX,NUM,SYM,X,INTJ
235,Lembrou-se do seu Byron,<p>\n\tA poesia de Lord Byron inspira a segund...,José Martiniano de Alencar – Senhora,annasalviato@hotmail.com,10,A poesia de Lord Byron inspira a segunda geraç...,"[[-9.197039, -10.817373, -10.572114, -12.23438...","[[-7.4030085, -8.012818, -9.072555, -9.461629,...","[[1.2726889, 2.8953836, -0.7336545, 1.0329155,...",2,...,3,1,1,1,3,1,1,1,1,1


In [ ]:
free_annotations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707 entries, 0 to 706
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   excerpt        707 non-null    object
 1   content        707 non-null    object
 2   clean_content  707 non-null    object
dtypes: object(3)
memory usage: 16.7+ KB


In [ ]:
free_annotations['clean_content']

235    A poesia de Lord Byron inspira a segunda geraç...
236    A personagem, em vários momentos da obra me fe...
237    Essa passagem me fez lembrar da obra "Romeu e ...
238    Achei curioso o fato de que o autor se refere ...
239    Aurélia com as duas características mais preza...
                             ...                        
700                 Inserir aqui o que se quer comentar.
701              Corpo da anotação que está sendo feita.
702                O curso de Letras é o melhor da UEMA.
703                        Qual o tipo de discurso aqui?
704                               exemplo de discurso...
Name: clean_content, Length: 385, dtype: object

In [ ]:
x = free_annotations.iloc[:,4:].drop(columns='clean_content').fillna(0)
x.to_csv('/content/drive/My Drive/TCC/csv/final/bert_with_ner.csv')
x.head()

,notas,DET,NOUN,ADP,PROPN,VERB,ADJ,PUNCT,CCONJ,PRON,...,ADV,SCONJ,AUX,NUM,SYM,X,INTJ,vectorized_content_bert,vectorized_excerpt_bert,NER
235,10.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,...,1.0,3.0,1.0,1.0,1.0,1.0,1.0,6823.067871,3521.516113,1.0
236,10.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,...,1.0,3.0,1.0,1.0,1.0,1.0,1.0,17138.296875,1859.887207,1.0
237,10.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,...,1.0,3.0,1.0,1.0,1.0,1.0,1.0,10728.886719,9572.370117,3.0
238,10.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,...,1.0,3.0,1.0,1.0,1.0,1.0,1.0,11468.213867,2361.022461,2.0
239,10.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,1.0,...,1.0,3.0,1.0,1.0,1.0,1.0,1.0,5652.828613,10673.510742,1.0


In [ ]:
x_train = x.sample(frac=0.8)
x_test = x.drop(x.index)

In [ ]:
X_x_ner = x_train.drop(columns=['notas']).to_numpy()
y_x_ner = x_train['notas'].to_numpy()

In [ ]:
# X_x_ner

In [ ]:
# y_bert_ner
# pd.DataFrame(X_x_ner)
grid_bert_ner = GridSearchCV(estimator=LinearRegression(), param_grid={}, scoring={'r2-score': make_scorer(r2_score)}, refit='r2-score', return_train_score=True, cv=2,error_score='raise')
grid_bert_ner.fit(X_x_ner,y_x_ner)

GridSearchCV(cv=2, error_score='raise', estimator=LinearRegression(),
             param_grid={}, refit='r2-score', return_train_score=True,
             scoring={'r2-score': make_scorer(r2_score)})

In [ ]:
# X_x_ner = x.drop(columns=['notas', 'vectorized_content', 'vectorized_excerpt'])
# y_x_ner = x['notas']
# y_bert_ner
# pd.DataFrame(X_x_ner)
# grid_bert_ner = GridSearchCV(estimator=LinearSVR(), param_grid={}, scoring={'r2-score': make_scorer(r2_score)}, refit='r2-score', return_train_score=True, cv=2,error_score='raise')
# grid_bert_ner.fit(X_x_ner,y_x_ner)

In [ ]:
# y = x.drop(columns='NER')
# y.head()
# y.to_csv('/content/drive/My Drive/TCC/csv/final/vectorized_without_ner.csv')

In [ ]:
free_annotations['NER'].head()

235    1.0
236    1.0
237    3.0
238    2.0
239    1.0
Name: NER, dtype: float64

In [ ]:
# free_annotations.to_csv('/content/drive/My Drive/TCC/csv/completo/cleaned_annotations.csv', index=False)

In [ ]:
# full_data = pd.merge(free_annotations, tudo, how='inner', on='annotation_id' ) # https://datacarpentry.org/python-socialsci/11-joins/index.html
# nome da coluna para fazer o merge: annotation_id

In [ ]:
# full_data.head()

,annotation_id,excerpt_x,title,type,content,ref_annotation,ref_user,excerpt_y,name,class_resource,label_class,label_domain,propertie_resource,prop_range,max_cardinality


In [ ]:
free_annotations['NER']

0                            []
1            [{'Jesus': 'PER'}]
2             [{'Deus': 'PER'}]
3                            []
4                            []
                 ...           
702         [{'Letras': 'LOC'}]
703                          []
704                          []
705    [{'Coelho Neto': 'PER'}]
706                          []
Name: NER, Length: 707, dtype: object